<a href="https://colab.research.google.com/github/omermehboob/NIDS-using-machine-learning-and-deep-learning/blob/main/DNN_2_(NIDS_BINARY_7_features).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Refined_data_set(label_encoding_7_features).csv')

In [4]:
df.shape

(1336498, 9)

In [5]:
df.head()

,Unnamed: 0,Bwd Packet Length Std,Flow Bytes/s,Total Length of Fwd Packets,Fwd Packet Length Std,Flow IAT Std,Flow IAT Min,Fwd IAT Total,Label
0,0,0.00000,2046,140.0,0.000000,7.124800e+04,3.0,29304.0,1
1,1,0.00000,7,30.0,0.000000,2.005113e+06,10.0,4016867.0,0
2,2,0.00000,2,12.0,3.464102,3.057131e+06,97.0,5318063.0,1
3,3,0.00000,1081081,64.0,0.000000,4.901360e+01,1.0,48.0,1
4,4,426.14309,57,2691.0,181.667939,4.191832e+06,39.0,117000000.0,1


In [6]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
df.head()

,Bwd Packet Length Std,Flow Bytes/s,Total Length of Fwd Packets,Fwd Packet Length Std,Flow IAT Std,Flow IAT Min,Fwd IAT Total,Label
0,0.00000,2046,140.0,0.000000,7.124800e+04,3.0,29304.0,1
1,0.00000,7,30.0,0.000000,2.005113e+06,10.0,4016867.0,0
2,0.00000,2,12.0,3.464102,3.057131e+06,97.0,5318063.0,1
3,0.00000,1081081,64.0,0.000000,4.901360e+01,1.0,48.0,1
4,426.14309,57,2691.0,181.667939,4.191832e+06,39.0,117000000.0,1


In [8]:
X=df[["Bwd Packet Length Std", "Flow Bytes/s", "Total Length of Fwd Packets", "Fwd Packet Length Std",
     "Flow IAT Std", "Flow IAT Min", "Fwd IAT Total"]]
y=df[["Label"]]

In [9]:
print(X.shape)
print(y.shape)

(1336498, 7)
(1336498, 1)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [11]:
scaler = Normalizer().fit(X_train)
X_train = scaler.transform(X_train)

scaler = Normalizer().fit(y_train)
y_train = scaler.transform(y_train)

In [12]:
model = Sequential()
model.add(Dense(1024,input_dim=7,activation='relu'))  
model.add(Dropout(0.10))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.10))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              8192      
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 768)               787200    
                                                                 
 dropout_1 (Dropout)         (None, 768)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 769       
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 796,161
Trainable params: 796,161
Non-trai

In [14]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=512, epochs=100)

Epoch 1/100
2089/2089 [==============================] - 14s 6ms/step - loss: 0.3358 - accuracy: 0.8579 - val_loss: 23668172.0000 - val_accuracy: 0.8257
Epoch 2/100
2089/2089 [==============================] - 8s 4ms/step - loss: 0.3268 - accuracy: 0.8612 - val_loss: 11051987.0000 - val_accuracy: 0.8252
Epoch 3/100
2089/2089 [==============================] - 8s 4ms/step - loss: 0.3187 - accuracy: 0.8641 - val_loss: 2154971.2500 - val_accuracy: 0.8507
Epoch 4/100
2089/2089 [==============================] - 8s 4ms/step - loss: 0.2892 - accuracy: 0.8719 - val_loss: 1703361.3750 - val_accuracy: 0.8386
Epoch 5/100
2089/2089 [==============================] - 8s 4ms/step - loss: 0.2442 - accuracy: 0.9006 - val_loss: 1648694.7500 - val_accuracy: 0.8438
Epoch 6/100
2089/2089 [==============================] - 8s 4ms/step - loss: 0.2219 - accuracy: 0.9112 - val_loss: 1361660.5000 - val_accuracy: 0.8413
Epoch 7/100
2089/2089 [==============================] - 8s 4ms/step - loss: 0.2146 - accur

In [15]:
from sklearn import metrics
def decode_sentiment(score):
    return 1 if score>0.5 else 0

In [16]:
scores = model.predict(X_test)
y_pred = [decode_sentiment(score) for score in scores]
accuracy=metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy of DNN_2 is {accuracy:.2f}%')

Accuracy of DNN_1 is 0.82%


In [17]:
print("classification report:")
print(metrics.classification_report(y_test, y_pred))

classification report:
              precision    recall  f1-score   support

           0       0.80      0.01      0.01     47065
           1       0.82      1.00      0.90    220235

    accuracy                           0.82    267300
   macro avg       0.81      0.50      0.46    267300
weighted avg       0.82      0.82      0.75    267300

